# P-values in ML 02

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb
import scipy as sc
import sklearn as sk
import sys
import os

from scipy.stats import chi2
from sklearn.ensemble import RandomForestClassifier
from statsmodels.discrete.discrete_model import Logit
from statsmodels.stats.multitest import fdrcorrection

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Create dataset

In [2]:
def logit(h):
    ''' Logistic from activation h '''
    p = 1.0 / (1.0 + np.exp(-h))
    r = np.random.rand(len(p))
    y = (r < p).astype('float')
    return y


def rand_date():
    max_time = int(time.time())
    t = random.randint(0, max_time)
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(t))


def rand_unif(num, mean, std, na_prob=0):
    xi = np.random.rand(num)
    if na_prob > 0:
        xi_na = (np.random.rand(num) <= na_prob)
        xi[xi_na] = np.nan
    return xi


def rand_norm(num, mean=0.0, std=1.0, na_prob=0):
    xi = np.random.normal(mean, std, num)
    if na_prob > 0:
        xi_na = (np.random.rand(num) <= na_prob)
        xi[xi_na] = np.nan
    return xi


def rbf(x, mu=0, sigma=1):
    """ Radial basis function """
    z = (x - mu) / sigma
    return np.exp(-(z*z))


def split_df(df, var_inputs, var_output, train_index, val_index):
    df_train, df_val = df.iloc[train_index], df.iloc[val_index]
    return df_train[var_inputs], df_train[var_output], df_val[var_inputs], df_val[var_output]

In [3]:
def create_dataset_01(num=1000, save=False):
    """ Create dataset y = f(x1, x2, x3) + noise (r* are not used) """
    # Inputs
    x1 = rand_norm(num)
    x2 = rand_norm(num)
    x3 = rand_norm(num)
    # Noise
    n1 = rand_norm(num)
    n2 = rand_norm(num)
    n3 = rand_norm(num)
    n = rand_norm(num)
    y = logit(2.0 * x1 - 1.0 * x2 + 0.5 * x3 + 0.25 * n)
    df = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'n1': n1, 'n2': n2, 'n3': n3, 'y': y})
    if save:
        file = 'zzz.csv'
        print(f"Saving dataset to file '{file}'")
        df.to_csv(file, index=False)
    return df

In [4]:
def create_dataset_02(num=1000, save=False):
    """ Create dataset y = f(x1, x2, x3) + noise (r* are not used) """
    # Inputs
    x1 = rand_norm(num)
    x2 = rand_norm(num)
    x3 = rand_norm(num)
    # Noise
    n1 = rand_norm(num)
    n2 = rand_norm(num)
    n3 = rand_norm(num)
    n = rand_norm(num)
    y = logit(2.0 * rbf(x1, 1, 1) - 1.0 * rbf(x2, -1, 1) + 0.5 * x3 + 0.25 * n)
    df = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'n1': n1, 'n2': n2, 'n3': n3, 'y': y})
    if save:
        file = 'zzz.csv'
        print(f"Saving dataset to file '{file}'")
        df.to_csv(file, index=False)
    return df

## Feature importance: Logistic regression

In [5]:
def wilks_model_fit(x, y):
    logit_model = Logit(y, x)
    res = logit_model.fit(disp=0)
    return logit_model, res

def wilks_p_value(df, var_output, vars_null):
    model_null, model_null_results = wilks_model_fit(df[vars_null], df[var_output])
    pvalues = dict()
    for c in df.columns:
        if c != var_output and c not in vars_null:
            xnames = list(vars_null)
            xnames.append(c)
            model_alt, model_alt_res = wilks_model_fit(df[xnames], df[var_output])
            if model_alt is None:
                self._error(f"Could not fit alt model for column/s {c}")
                pval = 1.0
            else:
                d = 2.0 * (model_alt_res.llf - model_null_results.llf)
                pval = chi2.sf(d, 1)
            pvalues[c] = pval
    return pd.Series(pvalues)

## Feature importance: ML

### Cross-validations & models

In [6]:
from sklearn.model_selection import KFold


def new_model_fit(x_train, y_train):
    model = RandomForestClassifier(100)
    model.fit(x_train, y_train)
    return model

_id = 0

def reset_id():
    _id = 0

def next_id():
    global _id
    _id += 1
    return _id


def is_var_rand(var):
    return var is not None and var.startswith('__rand_')


class FeatureImportanceShuffle:
    """ Feature importance by shuffling input values """
    def __init__(self, model, x_train, y_train, x_val, y_val, num_iter=10):
        self.id = next_id()
        self.model = model
        self.x_train, self.y_train, self.x_val, self.y_val = x_train, y_train, x_val, y_val
        self.num_iter = num_iter
        self.score_null = None  # Null score (one score)
        self.scores_alt = dict()  # Alt scores by variable name (multiple scores for each variable)

    def add_score(self, var, score):
        """ Add score for variable 'var' """
        if var not in self.scores_alt:
            self.scores_alt[var] = list()
        self.scores_alt[var].append(score)

    def __call__(self):
        """ Feature importance by shuffling variable 'var' and comparing performance results """
        if not self.score_null:
            self.score_null = self.model.score(self.x_val, self.y_val)
            # print(f"\t\tFeature importance (id={self.id}): Null model {self.scores_null}")
        for var in self.x_train.columns:
            # Perform shuffling 'num_iter' times
            scores = list()
            for i in range(self.num_iter):
                score = self.feature_importance_variable(var)
                scores.append(score)
                self.add_score(var, score)
            scores = np.array(scores)
            # print(f"\t{var}:\tcount: {len(scores)}\tmean: {scores.mean()}\tstd: {scores.std()}")

    def feature_importance_variable(self, var):
        """ Feature importance by shuffling variable 'var' and comparing performance results """
        # Shuffle 'var' and calculate validation score
        x_shuf = self.x_val.copy()
        x_shuf[var] = x_shuf[var].sample(frac=1).values
        score = self.model.score(x_shuf, self.y_val)
        # print(f"\t\tFeature importance (id={self.id}): Variable '{var}, score: {score}'")
        return score

    def get_null(self):
        """ Get null score """
        return self.score_null

    def get_null_and_rand(self, exclude_var=None):
        """ Get null and all '__rand_' scores as a list """
        values = [self.score_null]
        for var in self.x_train.columns:
            if is_var_rand(var) and var != exclude_var:
                values.extend(self.scores_alt[var])
        return values

    def get_alt(self, var):
        """ Get scores for variable 'var', or None if not found """
        return self.scores_alt.get(var)


class FeatureImportance:
    """
    Calculate feature importance.
    """
    def __init__(self, df, var_output, vars_input=None, vars_null=None, new_model_function=new_model_fit, num_cv=5, num_shuffle=10, num_rand=0, max_iter=100):
        self.df_ori = df
        self.var_output, self.vars_input, self.vars_null = var_output, vars_input, vars_null
        if self.vars_input is None:
            self.vars_input = [c for c in df.columns if c != var_output]
        if self.vars_null is None:
            self.vars_null = [c for c in self.vars_input if c.startswith('rand_')]
        self.new_model_function = new_model_function
        self.df = None
        self.pvals_df = None
        self.max_iter =max_iter
        self.num_cv = num_cv
        self.num_rand = num_rand
        self.num_shuffle = num_shuffle
        self.cvs = list()  # List of cross validation items
    
    def __call__(self):
        # initialize
        self.df = self.extend_df_ori()
        self.initialize_cv()
        # Calculate importance by shuffling
        for count_iter in range(self.max_iter):
            self.pvals_df = pd.DataFrame()
            for fis in self.cvs:
                print(f"Iteration: {count_iter}, Cross-validation: {fis.id}")
                fis()
            stop = self.p_values()
            if stop:
                return self.pvals_df
        return self.pvals_df
    
    def cv_iter(self, df):
        """ Create a cross-validation iterator """
        if self.num_cv > 1:
            return KFold(n_splits=self.num_cv).split(df)
        else:
            # No cross validation, split 80% / 20%
            idx = int(0.8 * len(df))
            idx_train = range(0, idx)
            idx_val = range(idx, len(df))
            return [(idx_train, idx_val)]
        
    def extend_df_ori(self):
        """ Extend the data frame by adding random permutations and random variables """
        df = self.df_ori.copy()
        # Create one additional 'rand' column for every variable
        for c in self.df_ori.columns:
            if c == self.var_output:
                continue
            c_rand = f"__rand_{c}"
            self.vars_input.append(c_rand)
            df[c_rand] = df[c].sample(frac=1).values
        # Create one additional 'rand' column for every variable
        for i in range(self.num_rand):
            c_rand = f"__rand_{i}"
            self.vars_input.append(c_rand)
            df[c_rand] = np.random.rand(len(df[c]))
        return df
        
    def get_alts(self, var):
        """ Get alt scores for variable 'var' """
        scores = list()
        for fis in self.cvs:
            ss = fis.get_alt(var)
            if ss is not None:
                scores.extend(ss)
        return np.array(scores)

    def get_nulls(self, var):
        """ Get null scores, excluding variable 'var' """
        scores = list()
        for fis in self.cvs:
            ss = fis.get_null_and_rand(exclude_var=var)
            if ss is not None:
                scores.extend(ss)
        return np.array(scores)

    def initialize_cv(self):
        """ Initialize cross validations """
        reset_id()
        for train_index, val_index in self.cv_iter(self.df):
            x_train, y_train, x_validate, y_validate = split_df(self.df, self.vars_input, self.var_output, train_index, val_index)
            model = self.new_model_function(x_train, y_train)
            fis = FeatureImportanceShuffle(model, x_train, y_train, x_validate, y_validate, num_iter=self.num_shuffle)
            self.cvs.append(fis)

    def p_value_mannwhitneyu(self, null_scores, alt_scores, alternative):
        try:
            return sc.stats.mannwhitneyu(null_scores, alt_scores, alternative=alternative)[1]
        except ValueError as ve:
            print(f"WARNING: Mann-Whitney U-test, {err}")
        finally:
            return 1.0

    def p_value(self, var, null_scores, alt_scores):
        null_scores, alt_scores = np.array(null_scores), np.array(alt_scores)
        pval_greater = sc.stats.mannwhitneyu(null_scores, alt_scores, alternative='greater')[1]
        pval_less = sc.stats.mannwhitneyu(null_scores, alt_scores, alternative='less')[1]
        pval_two = sc.stats.mannwhitneyu(null_scores, alt_scores, alternative='two-sided')[1]
        df_row = pd.DataFrame({'name': var
                               , 'num_samples': len(self.df_ori)
                               , 'num_cv': self.num_cv
                               , 'num_shuffle': self.num_shuffle
                               , 'count_alt': len(alt_scores)
                               , 'mean_alt': alt_scores.mean()
                               , 'std_alt': alt_scores.std()
                               , 'count_null': len(null_scores)
                               , 'mean_null': null_scores.mean()
                               , 'std_null': null_scores.std()
                               , 'p_value': pval_greater
                               , 'p_value_less': pval_less
                               , 'pval_two_sided': pval_two
                              }, index=[len(self.pvals_df)])
        self.pvals_df = self.pvals_df.append(df_row)
        return pval_greater

    def p_values(self):
        # Get null scores
        null_scores = list()
        for fis in self.cvs:
            null_scores.extend(fis.get_null_and_rand())
        # Calculate p-value for every variable
        pvals, isrand = list(), list()
        for var in self.vars_input:
            alt_scores = self.get_alts(var)
            null_scores = self.get_nulls(var)
            pval = self.p_value(var, null_scores, alt_scores)
            pvals.append(pval)
            isrand.append(is_var_rand(var))
        pvals = np.array(pvals)
        isrand = np.array(isrand)
        rejected, pvals_fdr = fdrcorrection(pvals)
        self.pvals_df['is_rand'] = isrand
        self.pvals_df['pvalues_fdr'] = pvals_fdr
        self.pvals_df['pvalues_fdr_reject'] = rejected
        return (rejected & isrand).any()


### P-values analysis

In [7]:
def pvalues_shuffle_analysis():
    pvals_df = None
    for dataset_num in [1, 2]:
        for num_samples in [50, 100, 200, 300, 500, 1000, 2000, 5000, 10000]:
            for num_cv in [1, 3, 5, 10, 20]:
                for num_shuffle in [3, 5, 10, 20, 50, 100]:
                    print(f"dataset_num: {dataset_num}\tnum_samples: {num_samples}\tnum_cv: {num_cv}\tnum_shuffle: {num_shuffle}")
                    df = create_dataset_01(num_samples) if dataset_num == 1 else create_dataset_02(num_samples)
                    fi = FeatureImportance(df, 'y', num_shuffle=num_shuffle, num_cv=num_cv)
                    pvals_df = fi()
                    pvals_df.to_csv(f"pvals_df.dataset_num_{dataset_num}.num_samples_{num_samples}.num_cv_{num_cv}.num_shuffle_{num_shuffle}.csv", index=False)
                    display(pvals_df)

                
pvalues_shuffle_analysis()

dataset_num: 1	num_samples: 50	num_cv: 1	num_shuffle: 3
Iteration: 0, Cross-validation: 1
Iteration: 1, Cross-validation: 1
Iteration: 2, Cross-validation: 1


,name,num_samples,num_cv,num_shuffle,count_alt,mean_alt,std_alt,count_null,mean_null,std_null,p_value,p_value_less,pval_two_sided,is_rand,pvalues_fdr,pvalues_fdr_reject
0,x1,50,1,3,9,0.577778,0.113312,55,0.770909,0.045418,2.351909e-06,0.999998,4.703819e-06,False,1.411146e-05,True
1,x2,50,1,3,9,0.733333,0.066667,55,0.770909,0.045418,4.207958e-02,0.960002,8.415915e-02,False,8.415915e-02,False
2,x3,50,1,3,9,0.500000,0.094281,55,0.770909,0.045418,5.402164e-08,1.000000,1.080433e-07,False,6.482597e-07,True
3,n1,50,1,3,9,0.788889,0.031427,55,0.770909,0.045418,8.719990e-01,0.133360,2.667193e-01,False,9.819763e-01,False
4,n2,50,1,3,9,0.711111,0.031427,55,0.770909,0.045418,3.410489e-04,0.999687,6.820977e-04,False,1.364195e-03,True
5,n3,50,1,3,9,0.800000,0.000000,55,0.770909,0.045418,9.690224e-01,0.032820,6.564062e-02,False,9.819763e-01,False
6,__rand_x1,50,1,3,9,0.788889,0.031427,46,0.767391,0.046878,9.034065e-01,0.101635,2.032691e-01,True,9.819763e-01,False
7,__rand_x2,50,1,3,9,0.788889,0.031427,46,0.767391,0.046878,9.034065e-01,0.101635,2.032691e-01,True,9.819763e-01,False
8,__rand_x3,50,1,3,9,0.766667,0.047140,46,0.771739,0.045027,3.862304e-01,0.624785,7.724609e-01,True,6.621093e-01,False
9,__rand_n1,50,1,3,9,0.733333,0.047140,46,0.778261,0.041247,3.739734e-03,0.996571,7.479468e-03,True,1.121920e-02,True


dataset_num: 1	num_samples: 50	num_cv: 1	num_shuffle: 5
Iteration: 0, Cross-validation: 2
Iteration: 1, Cross-validation: 2
Iteration: 2, Cross-validation: 2
Iteration: 3, Cross-validation: 2


,name,num_samples,num_cv,num_shuffle,count_alt,mean_alt,std_alt,count_null,mean_null,std_null,p_value,p_value_less,pval_two_sided,is_rand,pvalues_fdr,pvalues_fdr_reject
0,x1,50,1,5,20,0.535,1.458595e-01,121,0.608264,0.055427,7.495615e-03,0.992646,1.499123e-02,False,0.029299,True
1,x2,50,1,5,20,0.625,5.361903e-02,121,0.608264,0.055427,8.935498e-01,0.107778,2.155568e-01,False,0.998075,False
2,x3,50,1,5,20,0.595,6.689544e-02,121,0.608264,0.055427,1.818552e-01,0.820013,3.637104e-01,False,0.296680,False
3,n1,50,1,5,20,0.555,4.974937e-02,121,0.608264,0.055427,7.210772e-05,0.999930,1.442154e-04,False,0.000433,True
4,n2,50,1,5,20,0.515,7.921490e-02,121,0.608264,0.055427,3.183645e-07,1.000000,6.367290e-07,False,0.000004,True
5,n3,50,1,5,20,0.645,9.733961e-02,121,0.608264,0.055427,9.861341e-01,0.014108,2.821628e-02,False,0.998075,False
6,__rand_x1,50,1,5,20,0.580,7.483315e-02,101,0.613861,0.048807,9.766303e-03,0.990454,1.953261e-02,True,0.029299,True
7,__rand_x2,50,1,5,20,0.590,5.385165e-02,101,0.611881,0.055020,5.564960e-02,0.945303,1.112992e-01,True,0.133559,False
8,__rand_x3,50,1,5,20,0.640,4.898979e-02,101,0.601980,0.054464,9.974827e-01,0.002585,5.169592e-03,True,0.998075,False
9,__rand_n1,50,1,5,20,0.640,6.633250e-02,101,0.601980,0.050698,9.980750e-01,0.001978,3.955938e-03,True,0.998075,False


dataset_num: 1	num_samples: 50	num_cv: 1	num_shuffle: 10
Iteration: 0, Cross-validation: 3


ValueError: All numbers are identical in mannwhitneyu

# Improvement: Cluster high correlations (groups of variables)